In [5]:

"""Runs an XFOIL analysis for a given airfoil and flow conditions"""
import os
import subprocess
import numpy as np

# %% Inputs

def xfoil_analysis(airfoil_name, alpha_i, alpha_f, alpha_step, savepath, Re, n_iter = 100):

    #files will be stored in a seperate folder. First check if the folder exists if not create it.
 
    if not os.path.isdir(savepath):
        os.makedirs(savepath)

    savepath += "/" + str(airfoil_name) + ".txt"
    #file_name = str(airfoil_name) + ".txt"
 
    if os.path.exists(savepath):
        os.remove(savepath)

    if os.path.isfile(r"C:/Users/mtaha/OneDrive/Belgeler/GitHub/Panel_Method/input_file.in"):
        input_file = open(r"C:/Users/mtaha/OneDrive/Belgeler/GitHub/Panel_Method/input_file.in", 'w')
        # input_file.write("PLOP\n")
        # input_file.write("G, F\n")
        # input_file.write("\n\n")
        input_file.write("LOAD {0}.dat\n".format(airfoil_name))
        input_file.write(airfoil_name + '\n')
        input_file.write("PANE\n")
        input_file.write("OPER\n")
        input_file.write("Visc {0}\n".format(Re))
        input_file.write("PACC\n")
        input_file.write(savepath + "\n\n")
        input_file.write("ITER {0}\n".format(n_iter))
        input_file.write("ASeq {0} {1} {2}\n".format(alpha_i, alpha_f,
                                                    alpha_step))
        input_file.write("\n\n")
        input_file.write("quit\n")
        input_file.close()

        subprocess.call("C:/Users/mtaha/OneDrive/Belgeler/GitHub/Panel_Method/XFoil/xfoil.exe < input_file.in", shell=True)

        file_name = np.loadtxt(savepath, skiprows=12)
    else:
        print("File does not exist")


alpha_i = -5
alpha_f = 18
alpha_step = 1
n_iter = 100

ReS = [4*1e5, 5*1e5, 6*1e5, 7*1e5, 8*1e5, 9*1e5, 1e6]
for Re in ReS:
    Re = int(Re)
    
    folder_name = "NACAxxxx" + "_Re_" + str(Re)
    for i in range(0,7000):
        airfoil_name = "NACA" + str(i).zfill(4)
        if 11 + i//1000  < np.mod(i,100) < 40:
            xfoil_analysis(airfoil_name, alpha_i, alpha_f, alpha_step, folder_name, Re, n_iter)

    print("Re = ", Re, " is done")

Re =  400000  is done
Re =  500000  is done


KeyboardInterrupt: 

In [4]:
# This function reads the data from the NACAxxxx_Re_xxxxx.txt files and stores them in an array.

def read_data(airfoil_name, Re):
    
        folder_name = "NACAxxxx" + "_Re_" + str(Re)
        file_name = folder_name + "/" + airfoil_name + ".txt"
        data = np.loadtxt(file_name, skiprows=12)
        alpha = data[:,0]
        CL = data[:,1]
        CD = data[:,2]
        CM = data[:,4]
        return alpha, CL, CD, CM, Re

x_matrix = np.zeros((0, 5)) 
ReS = [1*1e5]
#, 2*1e5, 3*1e5, 4*1e5, 5*1e5, 6*1e5, 7*1e5, 8*1e5, 9*1e5, 1e6
y_matrix = np.zeros((0, 3))

for Re in ReS:
    Re = int(Re)
    for i in range(6,1310):
        airfoil_name = "NACA" + str(i).zfill(4)
        if 10 < np.mod(i,100) < 30:
                NAME = str(i).zfill(4)
                alpha, CL, CD, CM, Re = read_data(airfoil_name, Re)
                first_param = int(NAME[0])
                second_param = int(NAME[1])
                third_param = int(NAME[2:])
                numData = np.shape(alpha)[0]

                x = np.zeros((numData, 5))
                x[:,0] = first_param
                x[:,1] = second_param
                x[:,2] = third_param
                x[:,3] = Re
                x[:,4] = alpha
                x_matrix = np.vstack((x_matrix, x))
                y = np.zeros((numData, 3))
                y[:,0] = CL
                y[:,1] = CD
                y[:,2] = CM
                y_matrix = np.vstack((y_matrix, y))
               
CL_matrix = y_matrix[:,0]
CD_matrix = y_matrix[:,1]
CM_matrix = y_matrix[:,2]

np.savetxt("x_matrix.txt", x_matrix)
np.savetxt("CL_matrix.txt", CL_matrix)
np.savetxt("CD_matrix.txt", CD_matrix)
np.savetxt("CM_matrix.txt", CM_matrix)


                
                
        


In [3]:
# This function returns the CD and CL values for a given airfoil and flow conditions.

def get_CL_CD_CM(airfoil_name, Re, alpha):
        x_matrix = np.loadtxt("x_matrix.txt")
        CL_matrix = np.loadtxt("CL_matrix.txt")
        CD_matrix = np.loadtxt("CD_matrix.txt")
        CM_matrix = np.loadtxt("CM_matrix.txt")
        first_param = int(airfoil_name[4])
        second_param = int(airfoil_name[5])
        third_param = int(airfoil_name[6:])
        Re = int(Re)
        alpha = int(alpha)
        index = np.where((x_matrix[:,0] == first_param) & (x_matrix[:,1] == second_param) & (x_matrix[:,2] == third_param) & (x_matrix[:,3] == Re) & (x_matrix[:,4] == alpha))
        CL = CL_matrix[index]
        CD = CD_matrix[index]
        CM = CM_matrix[index]
        return CL, CD, CM

CL, CD, CM = get_CL_CD_CM("NACA0012", 100000, 5)